# **Imports**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer

# Data Normalization

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/final.csv')
print(df.shape)
df.head()

(10449, 20)


,id,artist,name,popularity,preview,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,label
0,2b8fOow8UzyDFAE27YhOZM,Maroon 5,Memories,99,NaN,0.764,0.320,11.0,-7.209,1.0,0.0546,0.8370,0.00000,0.0822,0.575,91.019,audio_features,189486.0,4.0,toplists
1,6WrI0LAC5M1Rw2MnX2ZvEg,Dua Lipa,Don't Start Now,95,https://p.scdn.co/mp3-preview/ed151225213380a4...,0.794,0.793,11.0,-4.521,0.0,0.0842,0.0125,0.00000,0.0952,0.677,123.941,audio_features,183290.0,4.0,toplists
2,3ZCTVFBt2Brf31RLEnCkWJ,Billie Eilish,everything i wanted,94,NaN,0.704,0.225,6.0,-14.454,0.0,0.0994,0.9020,0.65700,0.1060,0.243,120.006,audio_features,245426.0,4.0,toplists
3,696DnlkuDOXcMAnKlTgXXK,Arizona Zervas,ROXANNE,88,https://p.scdn.co/mp3-preview/17c8499c72603f1a...,0.621,0.601,6.0,-5.616,0.0,0.1480,0.0522,0.00000,0.4600,0.457,116.735,audio_features,163636.0,5.0,toplists
4,4VginDwYTP2eaHJzO0QMjG,Post Malone,Circles,68,NaN,0.704,0.758,0.0,-3.537,1.0,0.0402,0.2330,0.00144,0.0924,0.534,120.003,audio_features,214907.0,4.0,toplists


In [ ]:
cols = ['artist','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','label']
non_categorical = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']
categorical = ['artist','key','mode','time_signature','label']

# %ms = MinMaxScaler()
df[non_categorical] = df[non_categorical].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
# df[non_categorical] = mms.fit_transform(df[non_categorical])
# df.head()

# **Data Visualization**

In [ ]:
# corr = df[non_categorical].corr()
# corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# def cluster_plot(new_data):
#     kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
#     pred_y = kmeans.fit_predict(new_data)
#     t = [30,40,50,60,70,80,90]
#     label_color = [t[i] for i in pred_y]
#     plt.figure(figsize = (7,7))
#     plt.scatter(new_data.iloc[:,0],new_data.iloc[:,1], c=label_color, alpha=0.5,cmap='viridis')
#     plt.show()

# def silhouette(new_data):
#     kmeans = KMeans(random_state=0)
#     visualizer = KElbowVisualizer(kmeans, k=(2,11), metric='silhouette', timings=False)
#     visualizer.fit(new_data)
#     visualizer.poof()

In [ ]:
# cluster_plot(df[non_categorical])

# **PCA ( Principal Component Analysis)**

In [ ]:
pca = PCA(0.95)
data = df.drop_duplicates()
data = pca.fit_transform(data[non_categorical])
# cluster_plot(pd.DataFrame(data))
# pd.DataFrame(data).head()

In [ ]:
data = pd.DataFrame(data)

In [ ]:
# silhouette(data)

#### From the above graph we can determine that the clustering algorithm requires four clusters

# **Model Training**

In [ ]:
def train(df_train):
    n = 1
    for _ in range(n):
        # train_df, test_df = train_test_split(df_train)
        km = KMeans(
            n_clusters=4, init='random',
            n_init=10, max_iter=1000,
            tol=1e-04, random_state=0
        )
        y_km = km.fit(df_train)
        # km.labels_
        # km.predict(test_df)
    return km

def k_mean_distance(center_coordinates, data_coordiantes):
    summ=0
    mag=0
    for i in range(len(center_coordinates)):
        summ+=(center_coordinates[i]-data_coordiantes[i])**2
        mag+=(data_coordiantes[i])**2
    return (summ)*0.5

In [ ]:
km = train(data)
data = pd.DataFrame(data)
# pickle.dump(km, open('KMeans_Clustering', 'wb'))
data['label'] = km.labels_
data['artist'] = df.artist
data['name'] = df.name
data['preview'] = df.preview
data['popularity'] = df.popularity
data['type'] = df.label
data.head()

,0,1,2,3,4,5,6,label,artist,name,preview,popularity,type
0,0.256127,-0.521810,0.138744,-0.090820,0.127173,-0.112514,0.063021,2,Maroon 5,Memories,NaN,99,toplists
1,-0.542197,0.000944,0.128033,-0.145677,0.016477,-0.030308,-0.002892,1,Dua Lipa,Don't Start Now,https://p.scdn.co/mp3-preview/ed151225213380a4...,95,toplists
2,0.773966,0.020488,0.155532,-0.135814,0.148499,-0.179592,-0.091042,3,Billie Eilish,everything i wanted,NaN,94,toplists
3,-0.339391,0.045402,-0.145966,0.077931,0.211139,0.173919,-0.092936,1,Arizona Zervas,ROXANNE,https://p.scdn.co/mp3-preview/17c8499c72603f1a...,88,toplists
4,-0.325794,-0.065218,0.008516,-0.083025,-0.002374,-0.125450,0.046105,1,Post Malone,Circles,NaN,68,toplists


# **Generating Outcome**

In [ ]:
def song_recommendation(song,data):
    arr = []
    dummy_df = data.loc[data['label']==song.label.values[0]]
    #print(len(dummy_df.values))
    for i in range(len(dummy_df.values)):
        if(i>11): break
        dist = k_mean_distance(dummy_df.values[i][0:7],song.values[0][0:7])
        arr.append((
            dummy_df.values[i][11]/(dist+0.00000001)**2,
            dist,
            dummy_df.values[i][11],
            dummy_df.values[i][8],
            dummy_df.values[i][9],
            dummy_df.values[i][10],
            dummy_df.values[i][12]
        ))
    arr.sort()
    return arr

def song_print(song):
    print()
    print('='*100)
    print('Artist:         ', song.artist.values[0])
    print('Song Name:      ', song.name.values[0])
    print('Type:           ', song['type'].values[0])
    print('='*100)
    print()

# **Searching Songs by Name**

In [ ]:
def userselect():
    return 0

def song(user_input):
    song_list = []
    for k in df['name']:
        if (user_input).lower() in (k).lower():
            #print(k)
            song_list = df[df['name']==k].index.values
            #for _ in song_list:
            #    song = data.loc[[_]]
            #    print(song)
            break

    if len(song_list)==0:
        print("Other songs to try: ")
        print('-'*100)
        random_songs = data.sample(n=20)
        print(random_songs['name'])

        # print("Songs to try on: ",len(list(df.sample(n = 20)))
        # song = data.loc[[random.choice(df.sample(n=20))]]
        # ans = song_recommendation(song,data)
        # song_print(song)
        # j=1
        # for i in ans[::-1]:
        #     print('Number:               ', j)
        #     print('Popularity/distance:  ',i[0])
        #     print('Artist:               ', i[3])
        #     print('Song Name:            ', i[4])
        #     print('Type:                 ', i[6])
        #     print('-'*100)
          #     j+=1
    else:
        print('-'*100)
        print("Songs with similar name: ",len(song_list))
        song = data.loc[[random.choice(song_list)]]
        ans = song_recommendation(song,data)
        song_print(song)
        j=1
        for i in ans[::-1]:
            print('Number:               ', j)
            print('Popularity/distance:  ',i[0])
            print('Artist:               ', i[3])
            print('Song Name:            ', i[4])
            print('Type:                 ', i[6])
            print('-'*100)
            j+=1

if __name__ == '__main__':
    # user_input = input("Enter song name: ")
    song('abcdsvsvs')

Other songs to try: 
----------------------------------------------------------------------------------------------------
735                                 อนิจจัง (ไม่มีอีกแล้ว)
4103                                                  Ball
6839                                               Whip It
8988                                          Book Of Slim
3772                                       Let Me Be There
6281     I Don't Care (with Justin Bieber) - Loud Luxur...
5873                                                 Beach
6036                      GUD VIBRATIONS - Habstrakt Remix
3647                   Can't Get Enough Of Your Love, Babe
800                                                  Sleep
9856                                          Big Ole Love
3311                   Burning Down the House - 45 Version
9950                                      You're Beautiful
10299                  The Model - 2009 Remastered Version
8196                                      Lips Of An

# **Searching Songs by Artist name**

In [ ]:
user_input = input("Enter artist name: ")
artist_list = []
for k in df['artist']:
    if (user_input).lower() in (k).lower():
        #print(k)
        artist_list = df[df['artist']==k].index.values
        #for _ in song_list:
        #    song = data.loc[[_]]
        #    print(song)
        break

if len(artist_list)==0:
    print("Oops!!! Artist not found")
else:
    print('-'*100)
    print("Songs with similar artist name: ",len(artist_list))
    song = data.loc[[random.choice(artist_list)]]
    ans = song_recommendation(song,data)
    song_print(song)
    j=1
    for i in ans[::-1]:
        print('Number:               ', j)
        print('Popularity/distance:  ',i[0])
        print('Artist:               ', i[3])
        print('Song Name:            ', i[4])
        print('Type:                 ', i[6])
        print('-'*100)
        j+=1

Enter artist name: Elon
----------------------------------------------------------------------------------------------------
Songs with similar artist name:  1

Artist:          Felon
Song Name:       Crazy
Type:            edm_dance

Number:                1
Popularity/distance:   63001.672080265584
Artist:                Harry Styles
Song Name:             Lights Up
Type:                  toplists
----------------------------------------------------------------------------------------------------
Number:                2
Popularity/distance:   56536.5302556003
Artist:                Lauv
Song Name:             Mean It
Type:                  toplists
----------------------------------------------------------------------------------------------------
Number:                3
Popularity/distance:   43360.662967302356
Artist:                Halsey
Song Name:             Graveyard
Type:                  toplists
-----------------------------------------------------------------------------

# **Model Evaluation**

In [ ]:
# data_eval = data.iloc[:,0:7]

In [ ]:
# from yellowbrick.cluster import SilhouetteVisualizer
# kmeans = KMeans(4, random_state=42)
# visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
# plt.figure(figsize = (7,7))
# visualizer.fit(data_eval)        # Fit the data to the visualizer
# #visualizer.show()        # Finalize and render the figure

In [ ]:
# from yellowbrick.cluster import InterclusterDistance

# # Generate synthetic dataset with 12 random clusters
# # Instantiate the clustering model and visualizer
# model = KMeans(4, random_state=42)
# visualizer = InterclusterDistance(model)
# plt.figure(figsize = (10,10))
# visualizer.fit(data_eval)        # Fit the data to the visualizer
# #visualizer.show()        # Finalize and render the figure

## **Spotify Dataset Extraction**

In [ ]:
# import requests, json, base64
# clientId = "648371aa684041d49cf4416ef3f0aad7"
# clientSK = "5ff1e9388ea04affaf7cf996536f4b86"

# id_song = "11dFghVXANMlKmJXsNCbNl"


# class Spotify:

#     clientId = ""
#     clientSK = ""
#     token = ""
#     header = ""
#     base_url = "https://api.spotify.com/v1/"

#     def __init__(self, ci, csk):
#         self.clientId = ci
#         self.clientSK = csk
#         self.getToken()

#     def getToken(self):
#         url = "https://accounts.spotify.com/api/token"
#         auth = base64.b64encode((self.clientId + ":" + self.clientSK).encode("utf-8"))
#         header = {
#             "Authorization" : "Basic " + str(auth, "utf-8"),
#             "Content-type" : "application/x-www-form-urlencoded"
#         }
#         payload = {
#             "grant_type" : "client_credentials"
#         }
#         response = requests.post(url, headers=header, data=payload).json()
#         self.token = response["access_token"]
#         head =  {
#             "Authorization" : "Bearer " + self.token,
#         }
#         self.header = head

#     def getCategories(self, num = 20):
#         url = "https://api.spotify.com/v1/browse/categories?limit=20"
#         response = requests.get(url, headers=self.header).json()
#         cats = {}
#         for it in response["categories"]["items"]:
#             cats.update({it["id"] : it["href"]})
#         return cats

#     def getTrack(self):
#         url = "https://api.spotify.com/v1/audio-features/06AKEBrKUckW0KREUWRnvT"
#         response = requests.get(url, headers=self.header).json()
#         print(response)

#     def getPlaylists(self,category, num = 20):
#         url = "https://api.spotify.com/v1/browse/categories/" + category + "/playlists?limit=20"
#         response = requests.get(url, headers=self.header).json()
#         print(response)
#         plays = []
#         for it in response["playlists"]["items"]:
#             plays.append(it["id"])
#         return plays

#     def getPlaylistTracks(self,id):
#         url = "https://api.spotify.com/v1/playlists/" + id + "/tracks"
#         response = requests.get(url, headers=self.header).json()
#         tracks = []
#         for it in response["items"]:
#             try:
#                 tracks.append({"id":it["track"]["id"], "artist" : it["track"]["artists"][0]["name"], "name": it["track"]["name"], "popularity" : it["track"]["popularity"], "preview" :  it["track"]["preview_url"]})
#             except:
#                 pass
#         return tracks

#     def getTrackFeatures(self, track_arr):
#         url = "https://api.spotify.com/v1/audio-features"
#         queries = []
#         i =0
#         while i <= len(track_arr):
#             temp = track_arr[i: i + 100]
#             str = ",".join(temp)
#             queries.append(str)
#             i = i+ 100
#         # print(queries)
#         trackdetails = []
#         for items in queries:
#             url = "https://api.spotify.com/v1/audio-features/?ids=" + items
#             response = requests.get(url, headers=self.header).json()
#             trackdetails.extend(response["audio_features"])
#         return trackdetails


# spotify = Spotify(clientId,clientSK)

# print("connected")
# # data = ['2EyECcIQEsdFQoQRenrDcN', '5aGDIGKHrGAYJn33DsVdIF', '12NG9pyCWHHVU6qqi2eeY8', '1kCfP81Gj9wkbGkOpCkuZA', '7vIERuwVxmw24C1eKqbru7', '6lfMvOUpSYC0V2NFIsBLNI', '6C1fdB76jVoPCccAanw2dj', '6GeL3xpwOQ3wJu5qLMQJQc', '6QX9w5RIQvh20QrXuwbnFi', '3WNJcD4xvnHrt5q5RuzaeW', '02fMscVZ46N4l46yHZQkqT', '2MX3UFRtbMg7rqGxLjlP07', '3cn4bKSJShQzuoKMZ6XMis', '15SfsE8nukbTgfl8xu74d7', '3Uejh7R2Xq52oqrh8XOo5J', '0zMTiHXme9k2newg3Ir8Md', '7cw2yeC9wWZSuWwBjPTfwe', '6w938mZxCQfSjZRmwK329X', '32QACfMXvoSsiwxgulfIQU', '4QKV23A4GTDmZETzjtQI9c', '2JO3HwMRPeya8bXbtbyPcf', '0bHm2wFL0m7qZIlhO7PQu5', '6zQhANi5mDTELfSuUFPx4Q', '1i6N76fftMZhijOzFQ5ZtL', '2JzZ8UTEb1GGvR8Ra0Nfy2', '1y5LL9dpLXbCXNKKqw5wCJ', '0PDZ4Rgj36MxVgvoMv3sk5', '3yP0cohcr97BUNJcgvmSVg', '4OvQsAObGMF3dpkCV6DZzb', '1FL9DHDSED6lxNMDJUJQvB', '5p3JunprHCxClJjOmcLV8G', '5jzma6gCzYtKB1DbEwFZKH', '5moTxUGPZXgGmosl4rIELm', '16jUJ4RsmyQG8z3clJuUaz', '2UnY8ApZT4roi66n1LDAil', '7bx4zW72qdZKBb8p6g80jb', '78lgmZwycJ3nzsdgmPPGNx', '5ZgNecJcN9SSopnmCTlpXs', '5FTbeZMbUgKfaSaCN8bY09', '6HZ67VImxqr8aMBEEhblzf', '7bIQeY33PkduZRmR3h9BIY', '5YSI1311X8t31PBjkBG4CZ', '6Gi8vIT6In8RrT2eTKxoVR', '3W0xbvrZqaao3ZZEDhVQ0r', '4wZSxAXLNRFfQ0yyaCvHei', '0rme3FLbw5o1KLXpJzivjS', '0kqFlkmfvKzQ4QSmx5PZ1A', '6cFaRahEaLyew4ATx1QaUx', '1Qb2f8cDXorwKTbY0nQ0IH', '1aMxqU4b59JT5Y9Wtyusis', '0WNGPpmWqzPnk0psUhJ3SX', '6y4FVJwf09ssxuRnlEgXkp', '2hvOsGJc6qll4WzW8Ljqc3', '2QSD3K3b3BJ8DPhGhQfDPW', '21jBRxUvrWXTwBfGnQwyVy', '6FxD82u5P8yKohcYYDxyqw', '5QTxFnGygVM4jFQiBovmRo', '17GEcOhfo7rRr3wKumLHV2', '55TeO50ytH96i93yihdNJE', '1PwB9EADXS90I8LoewLXfl', '2h97II7QJZTBZv02HcocI7', '2yE3bwbhqypdsuhmv48Svn', '73HSIWZlSCfs4tqVkrGv7H', '7cdnq45E9aP2XDStHg5vd7', '0tsHAAhdxSzwHAeHHXKbsw', '6TNNMVpOgn8K5NoDC7alG6', '0ZfP7K8NoyJRjEfWWk8Mlv', '2sICTf5zrSZnTttimrcm2M', '0LzCSYjFxH3LKSU4UIeRBk', '4GRnCW4Alb5vcfptFytJdl', '4Ec8I6H20zu7F2vDdg90qP', '6RTh5BDes7y6KwC0rxW8WG', '0WghYc6JU0HbCvsgGlyqId', '3MFFDRC4wTN9JNGtzXsZlN', '5nYYu9UeWFZhjwFLJMfX77', '0ElRzK07sc9eszyk1ea9Ab', '3x0iCzjgf0v9lnwvAhU9P2', '3RZqcBz5WTWN2E5svMH162', '4Mj2GtsgUDaf43zxdw48hd', '3aInErz06eEAbRHMnsORR2', '3MmogIpfk1GVXRMmRjKLGI', '1wP1qnWy0cZWxnbWkzf9La', '0TaWdoJiAJoln9OQ1RbdNs', '00sydAz6PeOxYzwG1dRIPi', '7wLBPjReJhdkFtrGHgUcxG', '4lr8mubS2OpgJEOuPHx2rV', '0FnmRkP81O1I6oeOeU1xXS', '24M81fKyhTWBuYzczQMkOp', '5AdLu2shYmDMqt2pHaOEle', '6pXcyjABZyf6LKzL4wsj9k', '6e3qHQtKdbNhUVuuNNsyqD', '150NSrON3DeZM1aLyz3MoM', '5E5HYgxGMp3BPakHGfKfIB', '6ZWVeMysm8Q8zZu3lKnY2v', '7mxvuhmRrnQ31vHRm9A9qj', '2fWxcJeDQiLStdAdwF4ms6', '4l1Yx4GK7GU55Xtwl9we9U', '1vOgfMjWq10rW4h0U7yzO8', '5rJJnYeAeIsb0T4FkiaUvX', '4F797Kz0zqZiMxNcRdO8R2', '2rl3qVhTopu9De51tKCDUM', '2ngVZZGrIYAxyxaA2QcWTB', '65AABj5XsDcxGT92w3jK0o', '4wWQeafQ4q5WKafYIuUmY8', '0A8P76W8MXeulFGIHNWSG1', '716OZGLBg3vkNfMTpfbYm6', '0NyE3z63bQpiqDkx6DSjl0', '1ogk8gkNwRw9iHONH2CAna', '6z6i7ssJ5AIqSqhVcecJpa', '2XRT5pPoxTdudrcqfxHSvK', '01TyFEZu6mHbffsVfxgrFn', '14IN2apSi57uPG9hAC8kfL', '54H7CLmI5SE7wmhlE0THS5', '7b1aBkrngrTz8RXlkdQCay', '2DSxutganHw57vSdvmgjZt', '2Js2COGl7htc00CjZ2L2CO', '4soIPMs2XqUFu9roKupZ8I', '5P3JJGeG536cS1ciDnJsCk', '5KXPhIqiXPUBCxH3jyc10U', '5l4UVzmcW0vt2N7BXXS9M0', '41FUydVAeBuf0qLztk0d5j', '6jwZQIS9nPUQ7yCCouusqx', '5EUJ3wPl8KVTbsYgdsCixv', '66SPXY48GHqvedbRT7jBey', '66hEhjJ69nPkPyHCB5mANE', '5j375wUE1V53HpkKtiDKT2', '3HV9YnoYACjlrJ5CiZ3hLj', '46IEYjzyS37lltLLCDJbC0', '1B2SbIJqWWjMa7yGeeOWDp', '7eXDhsMpOFdsJWauZgq9qv', '1bnzho2GEO8gOg1j0a8Tnq', '5g6PqhkeTyARGPROM5Gosp', '6dDWdTZXJuDb5aMD3MC4eX', '4bMXxqBnfgHec86gvIJSb1', '0kBAFAVajRjCX8kXneVsZI', '5Pdsyk2NuTJ56mE97DmxmN', '2SIgKA7WTqDWQrcO8IP8Sc', '2Cw4Z95OdYBElmbdhj5EwU', '4QgIkSyVxv6FmsWO2dZz4J', '5jh147eWeuNyimT2Oo1A5J', '1pPesOxENcN3XdZGV3jOjs', '46YxybUOOH2MYRXBoHXUgw', '6jKDR323PynjvTkoWTNbqx', '4bFtkOCmHiRx6SjIAL1hdm', '7GfcvEme8J6sI3k8pNTU2Y', '7jqM0p5mYWqgwwVjdmWD8W', '2OGta1TI4EkWVR71xZBcfF', '7d9212orrSfVHQaSdesffl', '6D0RsrrwQ2YSYA9lv4lgz5', '2KeTrh78SDZCwwtmGlgncd', '5RyGi46ibm6UKnuDO5kpdO', '1l0CFINuN2Ug755WAxblxL', '4wf0EYJDLjaHoRXJ7fAvxm', '46Mq99aSEm1UoIg42pmCZm', '225WgsvbK6RoIH6Y7OhKP2', '19cprjSakswn0VsENCg6uu', '5I324DQSe589WK21l83G6X', '2OP0X9P8khKcUDtWUzCLHv', '1t3nMYYdEvbubaD5FMDglq', '4FDkX9JD3FzmttdfYI3JvS', '0K8N0F3efUhd55TxNGgmGD', '6PrTCr4WB1Pyf7Fk1Hd1Da', '75OoxBhx3J7ptBxmu4SqAi', '7cAkG4eP6QhXj3Sq3sdIfF', '71fC5JZdi0w1nTMnQomdsO', '5d9BthpFORCiN5S1lVGCSc', '68T6wCFarzNXU5dBywqc60', '5tJghWo8o7FYLtRDFpRg3R', '1Gij4Ja8h7W64VXi1Eqj6j', '21dr1PIP5wVs3OdpLaMkM3', '0cDpyfAXMCKgKEEnkYzb3D', '5rDzL33WiYowiXlZhwWvst', '2eN6V8hiaW2BB7ol4UVvYf', '0IhH5cuZdOg3eELZe4yVtk', '2s30KXFLxnyH9CHKoO3Yum', '0AACjaI0rxjufqxmsu6OfF', '3QNkkruVrn8sXLicRKg6P9', '5g2JXHcnhj6nmjzzE11G9P', '0v8UvAhruTvkA90MR7039n', '6urUaCWU3HXdalm0oZU1Cd', '7iJ5IuGJ7AX97KSrFfcREa', '2Y0xdEsWhfDAsvtMzFqJ73', '4Em5Dsf7FZheKNUuBvujzf', '6kCQsjFV3SubIrNTDkGrdr', '5zaVuOiil4YVgKLXEbya43']

# categories = spotify.getCategories(30)

# print("categories fetched")
# for key, value in categories.items():
#     categories[key] = spotify.getPlaylists(key,20)

# print("playlists fetched")
# print("write playlists to file")

# with open('play.txt', 'w') as file:
#     file.write(json.dumps(categories))

# data = []
# with open('play.txt') as f:
#     for line in f:
#         data.append(json.loads(line))

# categories = data[0]

# for key, value in categories.items():
#     tracks = []
#     for playlists in value:
#         tracks.extend(spotify.getPlaylistTracks(playlists))
#     categories[key] = tracks

# del categories["wellness"]

# for key, value in categories.items():
#     print(key, len(value))
#     if(len(value) == 0):
#         del categories[key]

# print("write categories file")
# with open('/content/drive/MyDrive/Datasets/cat.txt', 'w') as file:
#     file.write(json.dumps(categories))

# data = []
# with open('/content/drive/MyDrive/Datasets/cat.txt') as f:
#     for line in f:
#         data.append(json.loads(line))
# categories = data[0]

# for key, value in categories.items():
#     track = []
#     for tracks in value:
#         track.append(tracks["id"])
#     track_desc = spotify.getTrackFeatures(track)
#     print(len(value), len(track_desc))
#     for i in range(len(value)):
#         try:
#             value[i].update(track_desc[i])
#         except:
#             pass
# print("abc")

# with open('/content/drive/MyDrive/Datasets/final.txt', 'w') as file:
#      file.write(json.dumps(categories))

In [ ]:
# import csv
# data = []
# with open('/content/drive/MyDrive/Datasets/final.txt') as f:
#     for line in f:
#         data.append(json.loads(line))

# data = data[0]

# d = []
# for key, value in data.items():
#     for values in value:
#         values.update({"label": key})
#         d.append(values)

# # with open('mycsvfile.csv', 'w') as f:
# #     w = csv.DictWriter(f, my_dict.keys())
# #     w.writeheader()
# #     w.writerow(my_dict)


# f = open('/content/drive/MyDrive/Datasets/data_new.csv', 'w')
# w = csv.DictWriter(f, d[0].keys())
# w.writeheader()
# w.writerows(d)
# f.close()

# **Spotify Song Extraction**

In [ ]:
# import sys
# import os
# import json
# import spotipy
# import spotipy.util as sp_util
# from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOauthError
# SpotifyClientCredentials(client_id='648371aa684041d49cf4416ef3f0aad7', client_secret='5ff1e9388ea04affaf7cf996536f4b86')
# from spotipy.client import SpotifyException

# # !export SPOTIPY_CLIENT_ID='648371aa684041d49cf4416ef3f0aad7'
# # !export SPOTIPY_CLIENT_SECRET='5ff1e9388ea04affaf7cf996536f4b86'

# def print_header(message, length=30):
#     """
#     Given a message, print it with a buncha stars all header-like
#     :param message: The message you want to print
#     :param length: The number of stars you want to surround it
#     """
#     print('\n' + ('*' * length))
#     print(message)
#     print('*' * length)

# def print_audio_features_for_track(track, track_features):
#     """
#     Given a track and a features response, print out the desired audio features for that track
#     :param track:
#     :param track_features:
#     :return:
#     """
#     desired_features = [
#         'tempo',
#         'time_signature',
#         'key',
#         'mode',
#         'loudness',
#         'energy',
#         'danceability',
#         'acousticness',
#         'instrumentalness',
#         'liveness',
#         'speechiness',
#         'valence'
#     ]

#     print('\n  {}'.format(track_string(track)))
#     for feature in desired_features:
#         # Pull out the value of the feature from the features
#         feature_value = track_features.get(feature)

#         # If this feature is the key, convert it to a readable pitch
#         if feature == 'key':
#             feature_value = translate_key_to_pitch(feature_value)

#         # Print the feature value
#         print('    {}: {}'.format(feature, feature_value))


# def print_audio_analysis_for_track(track, track_analysis):
#     """
#     Given a track and a analysis response, print out the analysis JSON
#     :param track:
#     :param track_analysis:
#     :return:
#     """
#     print('\n  {}'.format(track_string(track)))
#     print(json.dumps(track_analysis, indent=2))


# def choose_tracks(tracks):
#     """
#     Given a list of tracks, list them on the console and let the user choose a
#     selection of them.
#     :return: A list of selected track objects
#     """
#     for i, track in enumerate(tracks):
#         print('  {}) {}'.format(i + 1, track_string(track)))

#     # Choose some tracks
#     track_choices = input('\nChoose some tracks (e.g 1,4,5,6,10): ')

#     # Turn the input into a list of integers
#     try:
#         track_choice_indexes = [int(choice.strip()) for choice in track_choices.split(',')]
#     except ValueError as e:
#         print('Error: Invalid input.')
#         return []

#     # Grab the tracks from our track list and return them
#     selected_tracks = [tracks[index - 1] for index in track_choice_indexes]
#     return selected_tracks

# # Define the scopes that we need access to
# # https://developer.spotify.com/web-api/using-scopes/
# scope = 'user-library-read playlist-read-private'

# ################################################################################
# # Authentication Functions
# ################################################################################
# def authenticate_client():
#     """
#     Using credentials from the environment variables, attempt to authenticate with the spotify web API.  If successful,
#     create a spotipy instance and return it.
#     :return: An authenticated Spotipy instance
#     """
#     try:
#         # Get an auth token for this user
#         client_credentials = SpotifyClientCredentials()

#         spotify = spotipy.Spotify(client_credentials_manager=client_credentials)
#         return spotify
#     except SpotifyOauthError as e:
#         print('API credentials not set.  Please see README for instructions on setting credentials.')
#         sys.exit(1)


# def authenticate_user():
#     """
#     Prompt the user for their username and authenticate them against the Spotify API.
#     (NOTE: You will have to paste the URL from your browser back into the terminal)
#     :return: (username, spotify) Where username is the user's username and spotify is an authenticated spotify (spotipy) client
#     """
#     # Prompt the user for their username
#     username = input('\nWhat is your Spotify username: ')

#     try:
#         # Get an auth token for this user
#         token = sp_util.prompt_for_user_token(username, scope=scope)

#         spotify = spotipy.Spotify(auth=token)
#         return username, spotify
#     except SpotifyException as e:
#         print('API credentials not set.  Please see README for instructions on setting credentials.')
#         sys.exit(1)
#     except SpotifyOauthError as e:
#         redirect_uri = os.environ.get('SPOTIPY_REDIRECT_URI')
#         if redirect_uri is not None:
#             print("""
#     Uh oh! It doesn't look like that URI was registered as a redirect URI for your application.
#     Please check to make sure that "{}" is listed as a Redirect URI and then Try again.'
#             """.format(redirect_uri))
#         else:
#             print("""
#     Uh oh! It doesn't look like you set a redirect URI for your application.  Please add
#     export SPOTIPY_REDIRECT_URI='http://localhost/'
#     to your `credentials.sh`, and then add "http://localhost/" as a Redirect URI in your Spotify Application page.
#     Once that's done, try again.'""")
#         sys.exit(1)

# scope = 'user-library-read playlist-read-private'

# ################################################################################
# # Main Demo Function
# ################################################################################

# def main():
#     """
#     Our main function that will get run when the program executes
#     """
#     print_header('Spotify Web API Demo App', length=50)

#     # Run our demo function
#     retry = True
#     while retry:
#         try:
#             print("""
# Let's get some audio data!  How would you like to choose your tracks:
#   1.) Search for a song
#   2.) Choose from your playlists
#   3.) Pick from your saved songs""")
#             program_choice = input('Choice: ')
#         except ValueError as e:
#             print('Error: Invalid input.')

#         spotify = None
#         selected_tracks = []
#         if program_choice == '1':
#             spotify = authenticate_client()
#             selected_tracks = search_track(spotify)
#         elif program_choice == '2':
#             username, spotify = authenticate_user()
#             selected_tracks = list_playlists(spotify, username)
#         elif program_choice == '3':
#             username, spotify = authenticate_user()
#             selected_tracks = list_library(spotify, username)

#         if selected_tracks:
#             try:
#                 print("""
# Great, we have {} {}!  What would you like to see for these tracks:
#     1.) Audio Features (High-Level)
#     2.) Audio Analysis (Low-Level)""".format(len(selected_tracks),
#         ("tracks" if len(selected_tracks) > 1 else "track")))
#                 display_choice = input('Choice: ')
#             except ValueError as e:
#                 print('Error: Invalid input.')

#             if display_choice == '1':
#                 get_audio_features(spotify, selected_tracks, pretty_print=True)
#             elif display_choice == '2':
#                 get_audio_analysis(spotify, selected_tracks, pretty_print=True)


#         # Prompt the user to run again
#         retry_input = input('\nRun the program again? (Y/N): ')
#         retry = retry_input.lower() == 'y'


# ################################################################################
# # API Fetch Functions
# ################################################################################

# def get_audio_features(spotify, tracks, pretty_print=False):
#     """
#     Given a list of tracks, get and print the audio features for those tracks!
#     :param spotify: An authenticated Spotipy instance
#     :param tracks: A list of track dictionaries
#     """
#     if not tracks:
#         print('No tracks provided.')
#         return

#     # Build a map of id->track so we can get the full track info later
#     track_map = {track.get('id'): track for track in tracks}

#     # Request the audio features for the chosen tracks (limited to 50)
#     print_header('Getting Audio Features...')
#     tracks_features_response = spotify.audio_features(tracks=track_map.keys())
#     track_features_map = {f.get('id'): f for f in tracks_features_response}

#     # Iterate through the features and print the track and info
#     if pretty_print:
#         for track_id, track_features in track_features_map.items():
#             # Print out the track info and audio features
#             track = track_map.get(track_id)
#             print_audio_features_for_track(track, track_features)

#     return track_features_map

# def get_audio_analysis(spotify, tracks, pretty_print=False):
#     """
#     Given a list of tracks, get and print the audio analysis for those tracks!
#     :param spotify: An authenticated Spotipy instance
#     :param tracks: A list of track dictionaries
#     """
#     if not tracks:
#         print('No tracks provided.')
#         return

#     # Build a map of id->track so we can get the full track info later
#     track_map = {track.get('id'): track for track in tracks}

#     # Request the audio analysis for each track -- one at a time since these
#     # can be really big
#     tracks_analysis = {}

#     print_header('Getting Audio Audio Analysis...')
#     for track_id in track_map.keys():
#         analysis = spotify.audio_analysis(track_id)
#         tracks_analysis[track_id] = analysis

#         # Print out the track info and audio features
#         if pretty_print:
#             track = track_map.get(track_id)
#             print_audio_analysis_for_track(track, analysis)

#     return tracks_analysis


# ################################################################################
# # Demo Functions
# ################################################################################

# def search_track(spotify):
#     """
#     This demo function will allow the user to search a song title and pick the song from a list in order to fetch
#     the audio features/analysis of it
#     :param spotify: An basic-authenticated spotipy client
#     """
#     keep_searching = True
#     selected_track = None

#     # Initialize Spotipy
#     # spotify = authenticate_client()

#     # We want to make sure the search is correct
#     while keep_searching:
#         search_term = input('\nWhat song would you like to search: ')

#         # Search spotify
#         results = spotify.search(search_term)
#         tracks = results.get('tracks', {}).get('items', [])

#         if len(tracks) == 0:
#             print_header('No results found for "{}"'.format(search_term))
#         else:
#             # Print the tracks
#             print_header('Search results for "{}"'.format(search_term))
#             for i, track in enumerate(tracks):
#                 print('  {}) {}'.format(i + 1, track_string(track)))

#         # Prompt the user for a track number, "s", or "c"
#         track_choice = input('\nChoose a track #, "s" to search again, or "c" to cancel: ')
#         try:
#             # Convert the input into an int and set the selected track
#             track_index = int(track_choice) - 1
#             selected_track = tracks[track_index]
#             keep_searching = False
#         except (ValueError, IndexError):
#             # We didn't get a number.  If the user didn't say 'retry', then exit.
#             if track_choice != 's':
#                 # Either invalid input or cancel
#                 if track_choice != 'c':
#                     print('Error: Invalid input.')
#                 keep_searching = False

#     # Quit if we don't have a selected track
#     if selected_track is None:
#         return

#     # Request the features for this track from the spotify API
#     get_audio_features(spotify, [selected_track])

#     return [selected_track]

# if __name__ == '__main__':
#     main()

In [ ]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
# spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
#     client_id='648371aa684041d49cf4416ef3f0aad7',
#     client_secret='5ff1e9388ea04affaf7cf996536f4b86'))

# results = spotify.artist_albums(birdy_uri, album_type='album')
# albums = results['items']
# while results['next']:
#     results = spotify.next(results)
#     albums.extend(results['items'])

# for album in albums:
#     print(album['name'])

In [ ]:
from os import listdir
from os.path import isfile, join
from collections import Counter
import pandas as pd
new_file = open("final2.csv",'w')
mypath = "/home/ss2402/Downloads/check/music_data/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# print(onlyfiles
list_rel = []
ght = 1
for fi in onlyfiles:
    annotated_response = mypath + fi
    column_names = ['SUBJECT']
    cereal_df = pd.read_csv(annotated_response)
    ght = ght + 1
    final = cereal_df.SUBJECT.to_list()
    list_rel.extend(final)
keys = list(Counter(list_rel).keys())
values = list(Counter(list_rel).values())
for i in range(len(keys)):
    new_file.write(str(keys[i])+','+str(values[i])+'\n')
new_file.close()